In [2]:
import csv
import math
import pandas as pd
import numpy as np
import nvector as nv
from geographiclib.geodesic import Geodesic
from datetime import datetime
import geopandas as gpd
from numbers import Number
from shapely.geometry import Point

import folium

#helper functions
def vector_azimuth(source,dest):
    res = Geodesic.WGS84.Inverse(source[0],source[1],dest[0],dest[1])
    #azimuth is in degrees
    azimuth = res['azi1']
    if azimuth < 0:
        azimuth = azimuth + 360
    return azimuth

#helper functions
def timedelta_hours(source_time,dest_time):
    timedelta = dest_time - source_time
    return timedelta.total_seconds()/3600

#helper functions
def vector_magnitude(source,dest,start,end):
    res = Geodesic.WGS84.Inverse(source[0],source[1],dest[0],dest[1])
    #distance is in meters
    distance = res['s12']
    time_spent = timedelta_hours(start,end)
    distance_traveled_miles = (distance/1000) * 0.621371
    if time_spent > 0:
        travel_speed = distance_traveled_miles/time_spent
        return travel_speed
    else:
        return None

#helper functions
def vector_endpoint(source,distance,azimuth):
    azimuth = np.rad2deg(azimuth)
    res = Geodesic.WGS84.Direct(source[0],source[1],azimuth,distance)
    return [res['lat2'],res['lon2']]

#helper function for use in map-reduce
def my_sum(x,y):
    return x+y

def east_west(obs):
    return obs['magnitude'] * math.sin(obs['azimuth'])

def north_south(obs):
    return obs['magnitude'] * math.cos(obs['azimuth'])

def location(unit_mvmt_data):
    return [unit_mvmt_data[1],unit_mvmt_data[2]]

def time_stamp(unit_mvmt_data):
    return unit_mvmt_data[0].to_pydatetime()

#returns a list of distance traveled, mean vector direction and velocity
def unit_distance_direction(unit_mvmt_data,end_time):
    observations = []
    for count in range(0,len(unit_mvmt_data)-1):
        obs = dict()
        start = location(unit_mvmt_data[count])
        end = location(unit_mvmt_data[count+1])
        start_time = time_stamp(unit_mvmt_data[count])
        end_time = time_stamp(unit_mvmt_data[count+1])
        
        azi = np.deg2rad(vector_azimuth(start,end))
        mag = vector_magnitude(start,end,start_time,end_time)
        if mag is not None:
            obs['azimuth'] = azi
            obs['magnitude'] = mag
            observations.append(obs)

    if len(observations) > 0:
        V_e = reduce(my_sum,map(east_west,observations))/len(observations)
        V_n = reduce(my_sum,map(north_south,observations))/len(observations)

        mean_vect_direction_temp_deg = np.rad2deg(math.atan2(V_e,V_n))

        if mean_vect_direction_temp_deg < 180:
            mean_vect_direction_temp_deg += 180
        if mean_vect_direction_temp_deg > 180:
            mean_vect_direction_temp_deg -= 180

        mean_vect_direction = np.deg2rad(mean_vect_direction_temp_deg)

        mean_vect_magnitude = math.sqrt(math.pow(V_e,2) + math.pow(V_n,2))

        total_time_spent = timedelta_hours(time_stamp(unit_mvmt_data[0]),
                                           end_time)
        distance_traveled_miles = mean_vect_magnitude * total_time_spent
        travel_speed = mean_vect_magnitude

        return [distance_traveled_miles,mean_vect_direction,travel_speed]
    else:
        return None

#extrapolated endpoint based on travel velocity and timestamp
def unit_mean_endpoint(unit_mvmt_data,end_time):
    source = location(unit_mvmt_data[0])
    vect_dist_dir = unit_distance_direction(unit_mvmt_data,end_time)
    if vect_dist_dir is not None:
        distance_miles = vect_dist_dir[0]
        distance_meters = distance_miles * 1609.34
        direction = vect_dist_dir[1]
        try:
            return vector_endpoint(source,distance_meters,direction)
        except:
            return None
    else:
        return None

#get overall mean destination
def all_unit_mean_dest(unit_dests):
    dest_lats = []
    dest_lons = []
    for lat,lon in unit_dests:
        dest_lats.append(lat)
        dest_lons.append(lon)
    points = nv.GeoPoint(latitude=dest_lats,longitude=dest_lons,degrees=True)
    nvectors = points.to_nvector()
    n_EM_E = nvectors.mean_horizontal_position()
    g_EM_E = n_EM_E.to_geo_point()
    lat, lon = g_EM_E.latitude_deg, g_EM_E.longitude_deg
    mean_dest = [lat[0],lon[0]]
    return mean_dest

#process the movement of an army unit and obtain it's origin 
#and extrapolated destination (based on average heading and speed)
#returns a pair of origin and extrapolated destination
def process_unit_mvmt(unit_timed_locs,end_time):
    #check if there are more than one locations
    if len(unit_timed_locs) > 1:
        orig = location(unit_timed_locs[0])
        dest = unit_mean_endpoint(unit_timed_locs,end_time)
        return [orig,dest]
    else:
        return [None,None]
    
#parse and load csv file into a Pandas DataFrame
filename = 'Division.csv'
csvfile = open(filename,'r')
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])

#get last timestamp
grouped_bydate = df.groupby('MAP_DATE')
end_time = grouped_bydate.size().last_valid_index().to_pydatetime()

#filter out just the unit ID, weight, date and location
#POINT_Y is the latitude, POINT_X the longitude
filter_data = df.loc[:,['Army_Group','Num_Name','MAP_DATE','Weight_D','POINT_X','POINT_Y']]

#drop all rows with NaN
filter_data.dropna(inplace=True)

#group the data by army unit
grouped_byunit = filter_data.groupby(['Army_Group','Num_Name'])

allunits_orig_dest = [[index]+process_unit_mvmt(
                                army_unit_data.sort_values(by='MAP_DATE').
                                loc[:,['MAP_DATE','POINT_Y','POINT_X']].
                                values.tolist(),end_time) for index,army_unit_data in grouped_byunit]

#convert to dataframe to drop those with None values
res_df = pd.DataFrame(allunits_orig_dest,columns=['unit','origin','dest'])
res_df.dropna(inplace=True)

overall_mean_dest = all_unit_mean_dest(res_df['dest'])

#create a map zoomed in around Smolensk
battle_map = folium.Map([54.78, 32.04],zoom_start=6)

#marker for overall destination
folium.Marker(overall_mean_dest,icon=folium.Icon(color='red')).add_to(battle_map)
    
#marker for each unit's origin, popup shows unit name
for record in res_df.itertuples():
    folium.CircleMarker(record.origin,
                        popup='-'.join(record.unit),
                        radius=4,
                        color='green').add_to(battle_map)
    origin_dest = [record.origin,overall_mean_dest]
    my_PolyLine=folium.PolyLine(locations=origin_dest,weight=1,color='black')
    battle_map.add_child(my_PolyLine)

battle_map

